In [ ]:
#creating visualizations

In [9]:
import pandas as pd
import numpy as np

In [7]:
#load and summarize the Wage and Cost Data
CostDF = pd.read_csv("CostDFwithPop.csv",index_col=0)
WageDF = pd.read_csv("WageDFwithPop.csv",index_col=0)

In [8]:
CostDF

,CostType,1 ADULT - 0,1 ADULT - 1,1 ADULT - 2,1 ADULT - 3,2 ADULTS(1 WORKING) - 0,2 ADULTS(1 WORKING) - 1,2 ADULTS(1 WORKING) - 2,2 ADULTS(1 WORKING) - 3,2 ADULTS(BOTH WORKING) - 0,2 ADULTS(BOTH WORKING) - 1,2 ADULTS(BOTH WORKING) - 2,2 ADULTS(BOTH WORKING) - 3,State,County,2022 Population Est
0,Food,"$3,351","$4,937","$7,417","$9,838","$6,143","$7,645","$9,856","$11,999","$6,143","$7,645","$9,856","$11,999",Alabama,Autauga,59759
1,Child Care,$0,"$6,261","$12,521","$18,782",$0,$0,$0,$0,$0,"$6,261","$12,521","$18,782",Alabama,Autauga,59759
2,Medical,"$2,733","$7,921","$7,708","$8,018","$6,298","$7,708","$8,018","$7,807","$6,298","$7,708","$8,018","$7,807",Alabama,Autauga,59759
3,Housing,"$8,310","$11,789","$11,789","$14,906","$9,946","$11,789","$11,789","$14,906","$9,946","$11,789","$11,789","$14,906",Alabama,Autauga,59759
4,Transportation,"$5,509","$9,776","$12,709","$15,012","$9,776","$12,709","$15,012","$16,304","$9,776","$12,709","$15,012","$16,304",Alabama,Autauga,59759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31695,Civic,"$2,581","$4,144","$5,120","$6,686","$4,144","$5,120","$6,686","$5,314","$4,144","$5,120","$6,686","$5,314",Wyoming,Weston,6860
31696,Other,"$4,748","$7,080","$6,477","$9,424","$7,080","$6,477","$9,424","$9,115","$7,080","$6,477","$9,424","$9,115",Wyoming,Weston,6860
31697,Required annual income after taxes,"$27,724","$53,555","$67,190","$87,748","$42,149","$51,688","$61,213","$65,933","$42,149","$59,575","$76,987","$89,594",Wyoming,Weston,6860
31698,Annual taxes,"$4,495","$8,682","$10,893","$14,225","$6,833","$8,379","$9,924","$10,689","$6,833","$9,658","$12,481","$14,524",Wyoming,Weston,6860


In [11]:
StateNames = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 
              'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 
              'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
              'Maine', 'Maryland', 'Massachusetts','Michigan', 'Minnesota',
              'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
              'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina',
              'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 
              'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 
              'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 
              'Wisconsin', 'Wyoming']

In [64]:
NewCostDF = pd.DataFrame(columns = CostDF.columns)
NewCostDF.drop(['County','2022 Population Est'],axis=1,inplace=True)
for state in StateNames:
    Dfstate = CostDF.where(CostDF['State'] == state).dropna(subset='State')
    StatePopSum = Dfstate.copy().drop_duplicates('County')
    SPSum = np.sum(StatePopSum['2022 Population Est'])
    CountyList = set(list(Dfstate['County']))
    NewStateDF = pd.DataFrame(columns = Dfstate.columns)
    NewStateDF.drop(['State','County','2022 Population Est'],axis=1,inplace=True)
    for county in CountyList:
        countyweight = StatePopSum.where(StatePopSum['County'] == county).dropna(subset='County')['2022 Population Est']
        countyweight = float(countyweight/SPSum)
        countyDF = Dfstate.where(Dfstate['County'] == county).dropna(subset='County')
        countyDF.drop(['State','County','2022 Population Est'],axis=1,inplace=True)
        toconcat = pd.DataFrame(columns = countyDF.columns)
        countyDF.set_index('CostType',inplace=True)
        for column in countyDF.columns:
            countyDF[column] = countyDF[column].apply(lambda x: str(x[1:]))
            countyDF[column] = countyDF[column].str.replace(',', '')
            countyDF[column] = countyDF[column].apply(lambda x: int(x)*countyweight)
        countyDF.reset_index(inplace=True) #was automatically named CostType
        toconcat = pd.concat([toconcat,countyDF])
        #print(toconcat)
        NewStateDF = pd.concat([NewStateDF,toconcat])
        #break
    NewStateDF = NewStateDF.groupby('CostType').sum()
    NewStateDF.reset_index(inplace=True)
    #noticed that middle group of 4 was missing the data that was the same in the left and right groups of 4:
    NewStateDF.at[1,'2 ADULTS(1 WORKING) - 1'] = NewStateDF.at[1,'1 ADULT - 1']
    NewStateDF.at[1,'2 ADULTS(1 WORKING) - 2'] = NewStateDF.at[1,'1 ADULT - 2']
    NewStateDF.at[1,'2 ADULTS(1 WORKING) - 3'] = NewStateDF.at[1,'1 ADULT - 3']
    NewStateDF['State'] = state
    NewCostDF = pd.concat([NewCostDF,NewStateDF])

In [65]:
NewCostDF.head()

,CostType,1 ADULT - 0,1 ADULT - 1,1 ADULT - 2,1 ADULT - 3,2 ADULTS(1 WORKING) - 0,2 ADULTS(1 WORKING) - 1,2 ADULTS(1 WORKING) - 2,2 ADULTS(1 WORKING) - 3,2 ADULTS(BOTH WORKING) - 0,2 ADULTS(BOTH WORKING) - 1,2 ADULTS(BOTH WORKING) - 2,2 ADULTS(BOTH WORKING) - 3,State
0,Annual taxes,6275.33906,11759.09926,14505.321522,18900.21873,9780.477044,11672.233022,13809.1871,14949.134397,9780.477044,13115.570583,16695.639045,19278.382915,Alabama
1,Child Care,0.0,6172.249776,12344.374006,18516.623782,0.0,6172.249776,12344.374006,18516.623782,0.0,6172.249776,12344.374006,18516.623782,Alabama
2,Civic,2503.0,4019.0,4965.0,6483.0,4019.0,4965.0,6483.0,5153.0,4019.0,4965.0,6483.0,5153.0,Alabama
3,Food,3351.0,4937.0,7417.0,9838.0,6143.0,7645.0,9856.0,11999.0,6143.0,7645.0,9856.0,11999.0,Alabama
4,Housing,8113.695503,10622.963089,10622.963089,13897.840658,8750.449581,10622.963089,10622.963089,13897.840658,8750.449581,10622.963089,10622.963089,13897.840658,Alabama


In [69]:
NewWageDF = pd.DataFrame(columns = WageDF.columns)
NewWageDF.drop(['County','2022 Population Est'],axis=1,inplace=True)
for state in StateNames:
    Dfstate = WageDF.where(WageDF['State'] == state).dropna(subset='State')
    StatePopSum = Dfstate.copy().drop_duplicates('County')
    SPSum = np.sum(StatePopSum['2022 Population Est'])
    CountyList = set(list(Dfstate['County']))
    NewStateDF = pd.DataFrame(columns = Dfstate.columns)
    NewStateDF.drop(['State','County','2022 Population Est'],axis=1,inplace=True)
    for county in CountyList:
        countyweight = StatePopSum.where(StatePopSum['County'] == county).dropna(subset='County')['2022 Population Est']
        countyweight = float(countyweight/SPSum)
        countyDF = Dfstate.where(Dfstate['County'] == county).dropna(subset='County')
        countyDF.drop(['State','County','2022 Population Est'],axis=1,inplace=True)
        toconcat = pd.DataFrame(columns = countyDF.columns)
        countyDF.set_index('WageType',inplace=True)
        for column in countyDF.columns:
            countyDF[column] = countyDF[column].apply(lambda x: str(x[1:]))
            countyDF[column] = countyDF[column].str.replace(',', '')
            countyDF[column] = countyDF[column].apply(lambda x: float(x.strip())*countyweight)
        countyDF.reset_index(inplace=True)
        toconcat = pd.concat([toconcat,countyDF])
        NewStateDF = pd.concat([NewStateDF,toconcat])
        #break
    NewStateDF = NewStateDF.groupby('WageType').sum()
    NewStateDF.reset_index(inplace=True)
    NewStateDF['State'] = state
    NewWageDF = pd.concat([NewWageDF,NewStateDF])

In [70]:
NewWageDF.head()

,WageType,1 ADULT - 0,1 ADULT - 1,1 ADULT - 2,1 ADULT - 3,2 ADULTS(1 WORKING) - 0,2 ADULTS(1 WORKING) - 1,2 ADULTS(1 WORKING) - 2,2 ADULTS(1 WORKING) - 3,2 ADULTS(BOTH WORKING) - 0,2 ADULTS(BOTH WORKING) - 1,2 ADULTS(BOTH WORKING) - 2,2 ADULTS(BOTH WORKING) - 3,State
0,Living Wage,15.920633,29.833772,36.800161,47.95106,24.813712,29.613171,35.035658,37.926495,12.408279,16.638972,21.178662,24.454083,Alabama
1,Minimum Wage,7.25,7.25,7.25,7.25,7.25,7.25,7.25,7.25,7.25,7.25,7.25,7.25,Alabama
2,Poverty Wage,6.19,8.38,10.56,12.74,8.38,10.56,12.74,14.92,4.19,5.28,6.37,7.46,Alabama
0,Living Wage,16.481,32.520618,40.82394,54.716877,25.44664,30.79343,36.116649,40.474174,12.723043,17.941728,23.15065,27.872535,Alaska
1,Minimum Wage,10.34,10.34,10.34,10.34,10.34,10.34,10.34,10.34,10.34,10.34,10.34,10.34,Alaska


In [71]:
NewWageDF.to_csv('NewWageDF.csv')
NewCostDF.to_csv('NewCostDF.csv')